In [95]:
import pandas as pd
train_df = pd.read_csv('data/train.csv', header=0)
test_df = pd.read_csv('data/test.csv', header=0)
y = train_df['Survived']
train_df = train_df.drop('Survived', axis=1)

In [96]:
from sklearn.base import BaseEstimator, TransformerMixin
class  DesiredAttributes(BaseEstimator, TransformerMixin):
    def fit (self, X, y=None):
        self.undesirable_columns = ['PassengerId','Name', 'Ticket', 'Cabin']
        return self

    def transform(self, X, y=None):
        return X.drop(self.undesirable_columns, axis=1)

In [121]:
class NumericAttributes(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.numericColumns = X.select_dtypes(include='number').columns
        return self
    def transform(self, X, y=None):
        return X[self.numericColumns]

numericAttributes = NumericAttributes()
Xnum = numericAttributes.fit_transform(Xdrop)
Xnum



,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500
...,...,...,...,...,...
886,2,27.0,0,0,13.0000
887,1,19.0,0,0,30.0000
888,3,NaN,1,2,23.4500
889,1,26.0,0,0,30.0000


In [120]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

pipenum = Pipeline([
    ('numericAttributes', NumericAttributes()),
    ('imputer', SimpleImputer(strategy='median')),
])

X_numeric_clean = pipenum.fit_transform(Xnum)
X_numeric_clean

array([[ 3.    , 22.    ,  1.    ,  0.    ,  7.25  ],
       [ 1.    , 38.    ,  1.    ,  0.    , 71.2833],
       [ 3.    , 26.    ,  0.    ,  0.    ,  7.925 ],
       ...,
       [ 3.    , 28.    ,  1.    ,  2.    , 23.45  ],
       [ 1.    , 26.    ,  0.    ,  0.    , 30.    ],
       [ 3.    , 32.    ,  0.    ,  0.    ,  7.75  ]])

In [99]:
class CategoricalAttributes(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.numericColumns = X.select_dtypes(include='object').columns
        return self
    def transform(self, X, y=None):
        return X[self.numericColumns]

In [100]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

pipecat = Pipeline([
    ('CategoricalAttributes', CategoricalAttributes()),
    ('imputer', SimpleImputer(strategy='most_frequent' )),
    ('enconder', OneHotEncoder())
])

In [101]:
from sklearn.pipeline import FeatureUnion, Pipeline

pipeunion = FeatureUnion([
    ('pipenum', pipenum),
    ('pipecat', pipecat)
 ])

In [102]:
from sklearn.pipeline import Pipeline

preprocessing = Pipeline([
    ('desiredAttributes', DesiredAttributes()),
    ('union', pipeunion)
])

In [122]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
import numpy as np


Xtrain = preprocessing.fit_transform(train_df)

model = DecisionTreeClassifier()
model.fit(Xtrain, y)
ypred = model.predict(Xtrain)

score = accuracy_score(y, ypred)
scores = cross_validate(model, Xtrain, y)
print(f"base de treino cross_validate={np.mean(scores['test_score'])}, accuracy={score}")

Xtest = preprocessing.fit_transform(test_df)
ypredtest = model.predict(Xtest)


print(f"base de treino cross_validate={np.mean(scores['test_score'])}")




base de treino cross_validate=0.7845521310652187, accuracy=0.9797979797979798
base de treino cross_validate=0.7845521310652187
